In [1]:

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from glob import glob
from keras import backend as K
sys.path.append("../NEAT")
from tqdm import tqdm
import cv2
from keras.models import Model, load_model
from NEATModels import yolo_loss, Concat
from stardist.models import StarDist2D
from csbdeep.models import Config, CARE
from csbdeep.utils import normalize
from NEATUtils import helpers, NMS, MultiModelPrediction
from NEATUtils.helpers import smallzero_pad, WatershedwithMask, fill_label_holes,MaxProjectDist, multiplot,Integer_to_border
import glob
from skimage.filters import threshold_otsu
from scipy.ndimage.morphology import binary_fill_holes
import numpy as np
from skimage.morphology import remove_small_objects
from tifffile import imread, imwrite
try:
    from pathlib import Path
    Path().expanduser()
except (ImportError,AttributeError):
    from pathlib2 import Path
try:
    import tempfile
    tempfile.TemporaryDirectory
except (ImportError,AttributeError):
    from backports import tempfile

Using TensorFlow backend.


In [2]:
#Path to your model directory
ModelDirectory = '/home/sancere/VarunNewton/CurieDeepLearningModels/O-NEATweights/'
#Path to timelapse image directory
MovieDirectory =  '/home/sancere/VarunCom/commun/Fl4-2-Var/Membrane/'
ResultsDirectory = MovieDirectory +  '/VarunResults/'

DenoisingModel = 'BorialisS1S2FlorisMidNoiseModel'
SegmentationMaskModel = 'DrosophilaMaskSegmentationCARE'
SegmentationStarModel = 'LucasDrosSmartSeeds'



In [3]:

Noisemodel = CARE(config = None, name = DenoisingModel, basedir = ModelDirectory)
model = StarDist2D(config = None, name = SegmentationStarModel, basedir = ModelDirectory)
UnetModel = CARE(config = None, name = SegmentationMaskModel, basedir = ModelDirectory)


Loading network weights from 'weights_best.h5'.
Loading network weights from 'weights_best.h5'.
Couldn't load thresholds from 'thresholds.json', using default values. (Call 'optimize_thresholds' to change that.)
Using default values: prob_thresh=0.5, nms_thresh=0.4.
Loading network weights from 'weights_best.h5'.


In [4]:
Path(ResultsDirectory).mkdir(exist_ok = True)
Path(ResultsDirectory + '/Mask/').mkdir(exist_ok = True)
Path(ResultsDirectory + '/Denoised/').mkdir(exist_ok = True)
Raw_path = os.path.join(MovieDirectory, '*tif')
X = glob.glob(Raw_path)
axes = 'ZYX'
axis_norm = (0,1)

for fname in X:
    image = imread(fname)
    Name = os.path.basename(os.path.splitext(fname)[0])
    print(Name)
    
    x = Noisemodel.predict(image, axes, n_tiles = (1, 4, 4))
    
    originalX = x.shape[1]
    originalY = x.shape[2]  

    #Get Name
    
    imwrite((ResultsDirectory + '/Denoised/' + Name+ '.tif' ) , x)
    #UNET Prediction
    Finalimage = np.zeros([x.shape[0], x.shape[1], x.shape[2]])
    Zseg = np.zeros([x.shape[0], x.shape[1], x.shape[2]])
    Volumeseg = np.zeros([x.shape[0], x.shape[1], x.shape[2]])
    for i in tqdm(range(0,x.shape[0])): 
                Segmented = UnetModel.predict(x[i,:],'YX')
                thresh = threshold_otsu(Segmented) 
                Binary = Segmented > thresh
                Finalimage[i,:] = Binary


                #Stardist Prediction

                x[i,:] = normalize(x[i,:],1,99.8,axis=axis_norm)
                x[i,:] = smallzero_pad(x[i,:], 64, 64)

                MidImage, details = model.predict_instances(x[i,:])
                StarImage = MidImage[:originalX, :originalY]

                smallprob, smalldist = model.predict(x[i,:])
                grid = model.config.grid
                midprob = cv2.resize(smallprob, dsize=(smallprob.shape[1] * grid[1] , smallprob.shape[0] * grid[0] ))
                middist = cv2.resize(smalldist, dsize=(smalldist.shape[1] * grid[1] , smalldist.shape[0] * grid[0] ))
                dist = MaxProjectDist(middist)
                prob = dist[:originalX, :originalY]
                Watershed, Markers = WatershedwithMask(prob,StarImage,Finalimage[i,:],  model.config.grid)
                Volumeseg[i,:] = Watershed
                Zseg[i,:] = Integer_to_border(Watershed)

    #Save different method segmentation

    imwrite((ResultsDirectory  + Name+ '.tif' ) , Zseg.astype('uint8'))
    imwrite((ResultsDirectory+ '/Mask/'  + Name+ '.tif' ) , Finalimage.astype('uint8'))
    imwrite((ResultsDirectory  + 'UnLabel' + Name + '.tif' ) , Volumeseg.astype('float32'))

    
    


Membrane-Dg-22-24_PHGFP-H2BRFP_21


100%|██████████| 60/60 [07:18<00:00,  7.31s/it]


In [5]:
from skimage.segmentation import  relabel_sequential
from skimage import measure
from scipy import spatial
def merge_labels_across_volume(labelvol, relabelfunc, threshold=3):
    nz, ny, nx = labelvol.shape
    res = np.zeros_like(labelvol)
    res[0,...] = labelvol[0,...]
    backup = labelvol.copy() # kapoors code modifies the input array
    for i in tqdm(range(nz-1)):
        res[i+1] = relabelfunc(res[i,...], labelvol[i+1,...],threshold=threshold)
        labelvol = backup.copy() # restore the input array
    return res

def RelabelZ(previousImage, currentImage,threshold):
    # This line ensures non-intersecting label sets
    currentImage = relabel_sequential(currentImage,offset=previousImage.max()+1)[0]
    # I also don't like modifying the input image, so we take a copy
    relabelimage = currentImage.copy()
    waterproperties = measure.regionprops(previousImage, previousImage)
    indices = [prop.centroid for prop in waterproperties] 
    labels = [prop.label for prop in waterproperties]
    if len(indices) > 0:
       tree = spatial.cKDTree(indices)
       currentwaterproperties = measure.regionprops(currentImage, currentImage)
       currentindices = [prop.centroid for prop in currentwaterproperties] 
       currentlabels = [prop.label for prop in currentwaterproperties] 
       if len(currentindices) > 0: 
           for i in range(0,len(currentindices)):
               index = currentindices[i]
               #print(f"index {index}")
               currentlabel = currentlabels[i] 
               #print(f"currentlabel {currentlabel}")
               if currentlabel > 0:
                      previouspoint = tree.query(index)
                      #print(f"prviouspoint {previouspoint}")
                      previouslabel = previousImage[int(indices[previouspoint[1]][0]), int(indices[previouspoint[1]][1])]
                      #print(f"previouslabels {previouslabel}")
                      if previouspoint[0] > threshold:
                             relabelimage[np.where(currentImage == currentlabel)] = currentlabel
                      else:
                             relabelimage[np.where(currentImage == currentlabel)] = previouslabel
    return relabelimage

In [6]:
Volumeseg = merge_labels_across_volume(Volumeseg.astype('uint16'), RelabelZ, threshold= 10)
imwrite((ResultsDirectory  + 'Label' + Name + '.tif' ) , Volumeseg.astype('float32'))

100%|██████████| 59/59 [01:36<00:00,  1.64s/it]


In [7]:
import napari
with napari.gui_qt():
   

    # initialise viewer with Left wing image
    viewer = napari.view_image(x, name='Raw', scale = [1,0.16,0.16])

    # add the labels
    label_layer = viewer.add_labels(Volumeseg, name='Volume', scale = [1,0.16,0.16]) 

In [9]:
Segmentation = '/home/sancere/VarunCom/commun/Fl4-2-Var/Membrane/VarunResults/UnLabelMembrane-Dg-22-24_PHGFP-H2BRFP_21.tif'
Volumeseg = merge_labels_across_volume(imread(Segmentation).astype('uint16'), RelabelZ, threshold= 10)
imwrite((ResultsDirectory  + 'Label' + Name + '.tif' ) , Volumeseg.astype('float32'))

100%|██████████| 59/59 [01:37<00:00,  1.66s/it]
